# Introduction

**Table of Contents**
1. Create a Simplified Agent Manually
    - 1.1. Create Manually a Simplified Agent without Memory
    - 1.2. Create Manually a Simplified Agent with Memory
2. Create an Agent via Built-in Methods
    - 2.1. Create an OpenAI Tools Agent

In [1]:
#!pip install python-dotenv \
#             langchain==0.2.5 \
#             langchain_core==0.2.9 \
#             langchain_groq==0.1.6
#             langchain_openai==0.1.16 \
#             wikipedia==1.4.0

In [2]:
# Change the current working directory to the pachage root
# That's step is due to the way settings.py is defined

import os


ROOT_DIR = os.path.join(*os.path.split(os.getcwd())[:-1])
os.chdir(ROOT_DIR)
os.getcwd()

'd:\\Projects\\agentic-rag-chatbot'

In [3]:
from dotenv import load_dotenv

conf_dir = os.path.join("conf")
conf_file = ".env"

conf_path = os.path.join(conf_dir, conf_file)
_ = load_dotenv(conf_path)

In [4]:
# Instantiate a callback function for propt printing

from langchain.callbacks.base import BaseCallbackHandler

class PromptPrintingCallbackHandler(BaseCallbackHandler):
    def __init__(self):
        pass

    def on_llm_start(self, serialized, prompts, **kwargs):
        for prompt in prompts:
            print(f"\n\n<-----\nPrompt being sent to the LLM API:\n{prompt}\n----->\n\n")

    def on_llm_end(self, result, **kwargs):
        pass

callback_handler = PromptPrintingCallbackHandler()


In [5]:
# Instatiate an LLM

model_provider = "openai" # "openai", "groq"

if model_provider=="groq":
    from langchain_groq import ChatGroq

    model_name = 'llama3-70b-8192'  # 'llama3-8b-8192', 'mixtral-8x7b-32768', 'llama3-70b-8192'
    model_kwargs = {
            'top_p': 0.95,
        }
    generate_kwargs = { 
            'max_tokens': 512,
            'temperature': 0.4,
    }
    groq_api_key = os.environ.get('GROQ_API_KEY', None)

    if groq_api_key is None:
        raise ValueError("GROQ_API_KEY is not set.")

    llm = ChatGroq(
        model_name=model_name, 
        groq_api_key=groq_api_key,
        model_kwargs=model_kwargs,
        **generate_kwargs,
        callbacks=[callback_handler],
    )

elif model_provider=="openai":
    from langchain_openai import ChatOpenAI

    model_name = "gpt-4o"
    model_kwargs = {
            'top_p': 0.95, 
        }
    generate_kwargs = { 
            'max_tokens': 512,
            'temperature': 0.4,
    }
    openai_api_key = os.environ.get('OPENAI_API_KEY', None)

    llm = ChatOpenAI(
        api_key=openai_api_key,
        model=model_name, 
        model_kwargs=model_kwargs,
        **generate_kwargs,
        callbacks=[callback_handler],
    )

In [6]:
# Example

llm.invoke("Hi there!")



<-----
Prompt being sent to the LLM API:
Human: Hi there!
----->




AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 10, 'total_tokens': 19}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_dd932ca5d1', 'finish_reason': 'stop', 'logprobs': None}, id='run-ffe37863-80da-4cf4-bae8-77bd9ca4201a-0', usage_metadata={'input_tokens': 10, 'output_tokens': 9, 'total_tokens': 19})

In [7]:
# Example 

# https://python.langchain.com/v0.2/docs/concepts/#chat-models

for chunk in llm.stream("what color is the sky?"):
    print(chunk.content, end="|", flush=True)



<-----
Prompt being sent to the LLM API:
Human: what color is the sky?
----->


|The| color| of| the| sky| can| vary| depending| on| several| factors|,| but| it| is| most| commonly| perceived| as| blue| during| a| clear| day|.| This| blue| color| is| due| to| Ray|leigh| scattering|,| where| shorter| blue| wavelengths| of| sunlight| are| scattered| in| all| directions| by| the| molecules| and| small| particles| in| the| Earth's| atmosphere|.| 

|At| sunrise| and| sunset|,| the| sky| can| take| on| shades| of| red|,| orange|,| pink|,| and| purple| due| to| the| scattering| of| light| passing| through| a| greater| thickness| of| the| atmosphere|,| which| filters| out| the| shorter| blue| wavelengths| and| allows| the| longer| red| wavelengths| to| dominate|.| 

|The| sky| can| also| appear| gray| or| white| on| cloudy| days|,| and| it| can| take| on| various| other| colors| under| different| atmospheric| conditions| or| during| certain| natural| phenomena|.||

In [8]:
# Tools (function to be called)

import datetime
import requests
import wikipedia

from langchain.agents import tool
from pydantic import BaseModel, Field

@tool
def search_wikipedia(query: str) -> str:
    """
    Searches Wikipedia for a given query and returns summaries of the top results.

    This function performs a search on Wikipedia for the specified query, retrieves the top three search results, 
    and returns a summary of each. If the retrieval fails for a particular page, it is skipped. If no summaries 
    are retrieved, it returns a message indicating no suitable results were found.

    Args:
        query (str): The search query to be used for retrieving information from Wikipedia.

    Returns:
        str: A string that concatenates the page title and summary of each of the top three search results,
             separated by double newlines. If no results are found or summaries are retrieved, a message is returned
             indicating the absence of good results.
    """
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[:3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except:
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)


@tool()
def search_ben_uri_gallery_and_museum(query: str) -> str:
    """
    Useful to retrieve content from Ben Uri Gallery and Museum website for a given query.

    Ben Uri Gallery and Museum's content focusing on the contributions of Jewish, Refugee, and Immigrant artists to 
    British visual culture.

    Args:
        query (str): The search query for which the information is to be retrieved from the Ben Uri gallery and museum.

    Returns:
        str: A string that concatenates the web page link and a significant text chunk for each of the top three results,
             separated by double newlines. If no relevant results are found, a message is returned indicating the absence 
             of useful information.
    """
    chunks = []
    try:
        results = benuri.search(query)
        for result in results[:3]:
            chunks.append(f"Page Link: {result.page_link}\nText Chunk: {result.content}")
    except:
        pass
    if not chunks:
        return "No good information was found in Ben Uri gallery & museum"
    return "\n\n".join(chunks)

tools = [search_wikipedia, search_ben_uri_gallery_and_museum]

In [9]:
# Example 

print(f'NAME:\n\n{search_wikipedia.name}\n\nDESCRIPTION:\n\n{search_wikipedia.description}\n\nARGUMENTS:\n\n{search_wikipedia.args}')

NAME:

search_wikipedia

DESCRIPTION:

Searches Wikipedia for a given query and returns summaries of the top results.

This function performs a search on Wikipedia for the specified query, retrieves the top three search results, 
and returns a summary of each. If the retrieval fails for a particular page, it is skipped. If no summaries 
are retrieved, it returns a message indicating no suitable results were found.

Args:
    query (str): The search query to be used for retrieving information from Wikipedia.

Returns:
    str: A string that concatenates the page title and summary of each of the top three search results,
         separated by double newlines. If no results are found or summaries are retrieved, a message is returned
         indicating the absence of good results.

ARGUMENTS:

{'query': {'title': 'Query', 'type': 'string'}}


# 1. Create a Simplified Agent Manually

## 1.1 Create Manually a Simplified Agent without Memory

#### Bind them to the LLM

In [10]:
# Get descriptions of the tools and bind them to the LLM
# https://python.langchain.com/v0.1/docs/modules/model_io/chat/function_calling/

llm_with_tools = llm.bind_tools(tools)

In [11]:
# Example

llm_with_tools

RunnableBinding(bound=ChatOpenAI(callbacks=[<__main__.PromptPrintingCallbackHandler object at 0x000001E27A546900>], client=<openai.resources.chat.completions.Completions object at 0x000001E27CFBA390>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001E27CFBB650>, model_name='gpt-4o', temperature=0.4, model_kwargs={'top_p': 0.95}, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=512), kwargs={'tools': [{'type': 'function', 'function': {'name': 'search_wikipedia', 'description': 'Searches Wikipedia for a given query and returns summaries of the top results.\n\nThis function performs a search on Wikipedia for the specified query, retrieves the top three search results, \nand returns a summary of each. If the retrieval fails for a particular page, it is skipped. If no summaries \nare retrieved, it returns a message indicating no suitable results were found.\n\nArgs:\n    query (str): The search query to be used for retrieving information 

In [12]:
# Example

llm_with_tools.invoke("Hi there! What's up?")



<-----
Prompt being sent to the LLM API:
Human: Hi there! What's up?
----->




AIMessage(content="Hello! I'm here and ready to assist you with any questions or tasks you have. How can I help you today?", response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 327, 'total_tokens': 352}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_d33f7b429e', 'finish_reason': 'stop', 'logprobs': None}, id='run-849b9463-a51e-4490-a1fb-e2a64862582a-0', usage_metadata={'input_tokens': 327, 'output_tokens': 25, 'total_tokens': 352})

In [13]:
# Example

llm_with_tools.invoke("tell me the most famous painting of the Jewish people")



<-----
Prompt being sent to the LLM API:
Human: tell me the most famous painting of the Jewish people
----->




AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uj7NmvIwDmtXRxJaOqdf8Idy', 'function': {'arguments': '{"query": "most famous Jewish painting"}', 'name': 'search_wikipedia'}, 'type': 'function'}, {'id': 'call_nHBjGlKSpUhSbuOIkTC04R64', 'function': {'arguments': '{"query": "most famous Jewish painting"}', 'name': 'search_ben_uri_gallery_and_museum'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 331, 'total_tokens': 388}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_d33f7b429e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-92ec2fed-8d4f-4bef-ab46-be68a5f7b274-0', tool_calls=[{'name': 'search_wikipedia', 'args': {'query': 'most famous Jewish painting'}, 'id': 'call_uj7NmvIwDmtXRxJaOqdf8Idy', 'type': 'tool_call'}, {'name': 'search_ben_uri_gallery_and_museum', 'args': {'query': 'most famous Jewish painting'}, 'id': 'call_nHBjGlKSpUhSbuOIkTC04R64', 'type': 'tool_call'}], usage_metadata={'i

In [14]:
# Example

llm_with_tools.invoke("what are contributions of Refugee to British visual culture?")



<-----
Prompt being sent to the LLM API:
Human: what are contributions of Refugee to British visual culture?
----->




AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_sBPdJv7zWfKjurIm8iOnYu31', 'function': {'arguments': '{"query": "contributions of refugees to British visual culture"}', 'name': 'search_wikipedia'}, 'type': 'function'}, {'id': 'call_GDsuZdRMA535idvANvvsyXlL', 'function': {'arguments': '{"query": "contributions of refugees to British visual culture"}', 'name': 'search_ben_uri_gallery_and_museum'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 332, 'total_tokens': 397}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_d33f7b429e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-49bb9d4f-0c1d-4e50-a3a7-02339208ba45-0', tool_calls=[{'name': 'search_wikipedia', 'args': {'query': 'contributions of refugees to British visual culture'}, 'id': 'call_sBPdJv7zWfKjurIm8iOnYu31', 'type': 'tool_call'}, {'name': 'search_ben_uri_gallery_and_museum', 'args': {'query': 'contributions of refugees to British vi

#### Define Prompt Template

In [15]:
from langchain.prompts import ChatPromptTemplate

system_message = """You are an AI assistant specialized in providing detailed information from the 
Ben Uri Gallery and Museum database. Your task is to use the 'search_ben_uri_gallery_and_museum' 
function to help users—students, researchers, academics, critics, art market professionals, and 
collectors—find accurate content and answers related to Jewish, Refugee, and Immigrant contributions 
to British visual culture."""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", "{input}"),
])

In [16]:
# Example

prompt.invoke("Hi there!")

ChatPromptValue(messages=[SystemMessage(content="You are an AI assistant specialized in providing detailed information from the \nBen Uri Gallery and Museum database. Your task is to use the 'search_ben_uri_gallery_and_museum' \nfunction to help users—students, researchers, academics, critics, art market professionals, and \ncollectors—find accurate content and answers related to Jewish, Refugee, and Immigrant contributions \nto British visual culture."), HumanMessage(content='Hi there!')])

#### Create the chain

In [17]:
chain = prompt | llm_with_tools

In [18]:
# Example

query = "tell me the most famous painting of the Jewish people"
response1 = chain.invoke({"input": query})
response1



<-----
Prompt being sent to the LLM API:
System: You are an AI assistant specialized in providing detailed information from the 
Ben Uri Gallery and Museum database. Your task is to use the 'search_ben_uri_gallery_and_museum' 
function to help users—students, researchers, academics, critics, art market professionals, and 
collectors—find accurate content and answers related to Jewish, Refugee, and Immigrant contributions 
to British visual culture.
Human: tell me the most famous painting of the Jewish people
----->




AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yddl8OGiNeEcCBT63zxggec4', 'function': {'arguments': '{"query":"most famous painting of the Jewish people"}', 'name': 'search_ben_uri_gallery_and_museum'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 412, 'total_tokens': 438}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_298125635f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3a36fb9f-01be-4f20-baee-775c83e9d42d-0', tool_calls=[{'name': 'search_ben_uri_gallery_and_museum', 'args': {'query': 'most famous painting of the Jewish people'}, 'id': 'call_yddl8OGiNeEcCBT63zxggec4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 412, 'output_tokens': 26, 'total_tokens': 438})

In [19]:
# Example

query = "what are contributions of Refugee to British visual culture?"
response2 = chain.invoke({"input": query})
response2



<-----
Prompt being sent to the LLM API:
System: You are an AI assistant specialized in providing detailed information from the 
Ben Uri Gallery and Museum database. Your task is to use the 'search_ben_uri_gallery_and_museum' 
function to help users—students, researchers, academics, critics, art market professionals, and 
collectors—find accurate content and answers related to Jewish, Refugee, and Immigrant contributions 
to British visual culture.
Human: what are contributions of Refugee to British visual culture?
----->




AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_QaLhOOddvsy4gtxFP6OctSQe', 'function': {'arguments': '{"query":"contributions of Refugee to British visual culture"}', 'name': 'search_ben_uri_gallery_and_museum'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 413, 'total_tokens': 441}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_298125635f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-70a2b992-46cf-49b8-bf70-65d1987b1277-0', tool_calls=[{'name': 'search_ben_uri_gallery_and_museum', 'args': {'query': 'contributions of Refugee to British visual culture'}, 'id': 'call_QaLhOOddvsy4gtxFP6OctSQe', 'type': 'tool_call'}], usage_metadata={'input_tokens': 413, 'output_tokens': 28, 'total_tokens': 441})

In [20]:
# Example

query = "what are the contributors to artificial inteligence?"
response3 = chain.invoke({"input": query})
response3



<-----
Prompt being sent to the LLM API:
System: You are an AI assistant specialized in providing detailed information from the 
Ben Uri Gallery and Museum database. Your task is to use the 'search_ben_uri_gallery_and_museum' 
function to help users—students, researchers, academics, critics, art market professionals, and 
collectors—find accurate content and answers related to Jewish, Refugee, and Immigrant contributions 
to British visual culture.
Human: what are the contributors to artificial inteligence?
----->




AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_i04J74dqKH1i3lrwagVH0VQh', 'function': {'arguments': '{"query": "Contributors to Artificial Intelligence"}', 'name': 'search_wikipedia'}, 'type': 'function'}, {'id': 'call_OGWpwVsdLncvH2upY2eT3kYU', 'function': {'arguments': '{"query": "Contributors to Artificial Intelligence"}', 'name': 'search_ben_uri_gallery_and_museum'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 411, 'total_tokens': 470}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_d33f7b429e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ad3e08c4-6c9d-48d1-aa0d-821b193268cb-0', tool_calls=[{'name': 'search_wikipedia', 'args': {'query': 'Contributors to Artificial Intelligence'}, 'id': 'call_i04J74dqKH1i3lrwagVH0VQh', 'type': 'tool_call'}, {'name': 'search_ben_uri_gallery_and_museum', 'args': {'query': 'Contributors to Artificial Intelligence'}, 'id': 'call_OGWpwVsdLncvH2upY2eT

## 1.2 Create Manually a Simplified Agent with Memory

In [21]:
import json

from langchain.memory.chat_memory import BaseChatMemory
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate
)
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.messages import ToolMessage
from langchain_core.messages.ai import AIMessage
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.system import SystemMessage
from langchain_core.tools import StructuredTool
from langchain_openai.chat_models.base import ChatOpenAI
from typing import Union, List, Dict, Any

class ConversationChain:
    """
    A class to manage and conduct a conversation chain with an AI, incorporating memory and tools.

    Attributes:
        llm (Union[ChatOpenAI, BaseChatModel]): The language model used for generating responses.
        tools (List[StructuredTool]): Tools that can be utilized by the language model.
        memory (BaseChatMemory): Manages the history and state of the conversation.
            Default value: ConversationBufferMemory
        system_message (str): An initial message describing the interaction's context.
            Default value: The following is a conversation between a human and an AI assistant. 
                           The assistant is helpful, creative, clever, and very friendly.
        verbose (bool): Flag to enable detailed logging for debugging.

    Methods:
        _initialize_chain(): Sets up the conversation templates and bindings.
        invoke(user_input): Processes the user's input and generates the AI's response.
        execute_tools(tool_calls): Executes specific tool functionalities based on AI response.
    """
    def __init__(self,
                 llm: Union[ChatOpenAI, BaseChatModel],
                 tools: List[StructuredTool],
                 memory: BaseChatMemory = ConversationBufferMemory(return_messages=True, memory_key='history'),
                 system_message: str = """The following is a conversation between a human and an AI assistant. 
                                          The assistant is helpful, creative, clever, and very friendly.""",
                 verbose: bool = False) -> None:
        self.llm = llm
        self.tools = tools
        self.memory = memory
        self.system_message = system_message
        self.verbose = verbose
        self._initialize_chain()

    def _initialize_chain(self) -> None:
        """Private method to initialize the chat prompt and chain with tools."""
        self.llm_with_tools = self.llm.bind_tools(self.tools)
        self.chat_prompt = ChatPromptTemplate(
            messages=[
                SystemMessagePromptTemplate.from_template(
                    f"""{self.system_message}\nConversation context:"""
                ),
                MessagesPlaceholder(variable_name="history"),
                HumanMessagePromptTemplate.from_template("{user_input}")
            ]
        )
        self.llm_chain = self.chat_prompt | self.llm_with_tools
        self.tool_mapping = {tool.func.__name__: tool.func for tool in self.tools}

    def invoke(self, user_input: str) -> Any:
        """
        Process user input to generate a response using the language model and tools.

        Args:
        user_input (str): The user's input text.

        Returns:
        Any: The response from the language model or an error if one occurred.
        """
        history = self.memory.load_memory_variables({})['history']
        try:
            ai_response = self.llm_chain.invoke({"history": history, "user_input": user_input})

            self.memory.chat_memory.messages.append(HumanMessage(user_input))
            self.memory.chat_memory.messages.append(ai_response)

            if ai_response.tool_calls:
                tool_results = self.execute_tools(ai_response.tool_calls)
                for tool_result in tool_results:
                    self.memory.chat_memory.messages.append(ToolMessage(**tool_result))
                # Note: It's being used llm (without tools) to avoid loops
                ai_response = self.llm.invoke(self.memory.chat_memory.messages)

                self.memory.chat_memory.messages.append(ai_response)

            return ai_response
        except Exception as e:
            return e

    def execute_tools(self, tool_calls: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Execute tools based on tool calls provided by the AI response.

        Args:
        tool_calls (List[Dict[str, Any]]): List of tool calls including tool name and arguments.

        Returns:
        List[Dict[str, Any]]: List of results from executing the tools.
        """
        tool_results = []
        for tool_call in tool_calls:
            tool_name = tool_call['name']
            tool_output = self.tool_mapping[tool_name](**tool_call['args'])
            tool_results.append({'tool_call_id': tool_call['id'], 'name': tool_name, 'content': tool_output})
        return tool_results

In [22]:
# Example

system_message = """You are an AI assistant specialized in providing detailed information from the 
Ben Uri Gallery and Museum database. Your task is to use the 'search_ben_uri_gallery_and_museum' 
function to help users—students, researchers, academics, critics, art market professionals, and 
collectors find accurate content and answers related to Jewish, Refugee, and Immigrant contributions 
to British visual culture. If you are not sure whether you should use the tool or respond based on 
your internal knowledge, ask for further clarification."""

chatbot = ConversationChain(llm=llm, tools=tools, system_message=system_message)

user_inputs = [
    "Hi there, my name is Luis. What's yours?",
    "tell me the most famous painting of the Jewish people"
    "what are contributions of Refugee to British visual culture?",
    "what are the contributors to artificial inteligence?"
]

for user_input in user_inputs:
    ai_answer = chatbot.invoke(user_input)
    print(ai_answer)



<-----
Prompt being sent to the LLM API:
System: You are an AI assistant specialized in providing detailed information from the 
Ben Uri Gallery and Museum database. Your task is to use the 'search_ben_uri_gallery_and_museum' 
function to help users—students, researchers, academics, critics, art market professionals, and 
collectors find accurate content and answers related to Jewish, Refugee, and Immigrant contributions 
to British visual culture. If you are not sure whether you should use the tool or respond based on 
your internal knowledge, ask for further clarification.
Conversation context:
Human: Hi there, my name is Luis. What's yours?
----->


content="Hello Luis! I'm an AI assistant here to help you with information related to the Ben Uri Gallery and Museum. How can I assist you today?" response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 440, 'total_tokens': 470}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_d33f7b429e', 'finish_reas

In [23]:
# Example

import pandas as pd

from langchain_core.messages import ToolMessage
from langchain_core.messages.ai import AIMessage
from langchain_core.messages.human import HumanMessage

chat_messages = chatbot.memory.chat_memory.messages.copy()

def get_role(message):
    if isinstance(message, HumanMessage):
        return 'Human'
    elif isinstance(message, AIMessage):
        return 'AI'
    elif isinstance(message, ToolMessage):
        return 'Tool'
    else:
        return 'Unkown'

def get_content(message):
    if message.content:
        return message.content
    else:
        return message.tool_calls

role = []
message = []
for chat_message in chat_messages:
    role.append(get_role(chat_message))
    message.append(get_content(chat_message))
data = {'role': role, 'message': message}


def highlight_max(row):
    is_ai = max(row == 'AI')
    is_tool = max(row == 'Tool')
    if is_ai:
        color = "green"
    elif is_tool:
        color = "gray"
    else:
        color = "black"
    return [f"background-color: {color}" for cel in row]


chat_history_formatted = (
    pd.DataFrame(data)
    .style.apply(highlight_max, axis=1)
)

chat_history_formatted

,role,message
0,Human,"Hi there, my name is Luis. What's yours?"
1,AI,Hello Luis! I'm an AI assistant here to help you with information related to the Ben Uri Gallery and Museum. How can I assist you today?
2,Human,tell me the most famous painting of the Jewish peoplewhat are contributions of Refugee to British visual culture?
3,AI,"It looks like you have two questions here. To provide you with the most accurate and detailed information, I will perform searches related to each query. 1. The most famous painting of the Jewish people. 2. Contributions of refugees to British visual culture. I'll start by searching the Ben Uri Gallery and Museum database for information on these topics. Please hold on for a moment while I gather the information."
4,Tool,No good information was found in Ben Uri gallery & museum
5,Tool,No good information was found in Ben Uri gallery & museum
6,AI,"It seems I couldn't find specific information directly from the Ben Uri Gallery and Museum database regarding the most famous painting of the Jewish people or the contributions of refugees to British visual culture. However, I can still provide some general insights on these topics. ### Most Famous Painting of the Jewish People When considering famous paintings related to Jewish culture and history, several works come to mind: 1. **""Marc Chagall's White Crucifixion""**: This painting is one of Marc Chagall's most famous works and depicts a crucified Jesus surrounded by scenes of Jewish suffering. It is a powerful commentary on the persecution of Jews, particularly during the Holocaust. 2. **""The Ghetto Fighters"" by Samuel Bak**: Samuel Bak is a Holocaust survivor whose works often reflect his experiences and the broader Jewish experience during and after World War II. ### Contributions of Refugees to British Visual Culture Refugees have made significant contributions to British visual culture, particularly during the 20th century. Here are a few notable examples: 1. **Lucian Freud**: Born in Berlin, Freud fled to Britain in 1933 to escape the rise of the Nazis. He became one of the most important British painters of the 20th century, known for his intense and detailed portraits. 2. **Frank Auerbach**: Another refugee from Nazi Germany, Auerbach arrived in Britain in 1939. His expressive and textured paintings have made him a leading figure in British art. 3. **Oskar Kokoschka**: An Austrian artist who fled to Britain during World War II, Kokoschka's works have had a lasting impact on British and European art. 4. **Ben Uri Gallery and Museum**: The gallery itself was founded by Jewish immigrants and has played a crucial role in promoting the works of Jewish and immigrant artists in Britain. These artists and institutions have enriched British visual culture with their diverse perspectives and experiences, often reflecting themes of displacement, identity, and resilience. If you have any more specific questions or need further details, feel free to ask!"
7,Human,what are the contributors to artificial inteligence?
8,AI,"Artificial Intelligence (AI) is a multidisciplinary field that has seen contributions from a wide array of scientists, researchers, and engineers. Here are some of the key contributors and their contributions to the development of AI: ### Early Pioneers 1. **Alan Turing**: Often considered the father of computer science and AI, Turing proposed the concept of a ""universal machine"" (the Turing Machine) and the famous Turing Test to measure a machine's ability to exhibit intelligent behavior. 2. **John McCarthy**: Coined the term ""Artificial Intelligence"" and was a key figure in the development of AI as a field. He also developed the Lisp programming language, which became a standard in AI research. 3. **Marvin Minsky**: Co-founder of the MIT Media Lab and a pioneer in AI, Minsky made significant contributions to the fields of robotics and cognitive psychology. 4. **Herbert A. Simon**: A Nobel laureate who made substantial contributions 

# 2. Create an Agent via Built-in Methods
Allowing the LLM to reason and use tools how many times it thinks that is necessary.

## 2.1. Create an OpenAI Tools Agent

In [24]:
# Define a prompt template

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate
)

system_message = """You are an AI assistant specialized in providing detailed information from the \
Ben Uri Gallery and Museum database. Your task is to use the 'search_ben_uri_gallery_and_museum' \
function to help users—students, researchers, academics, critics, art market professionals, and \
collectors find accurate content and answers related to Jewish, Refugee, and Immigrant contributions \
to British visual culture. If you are not sure whether you should use the tool or respond based on \
your internal knowledge, ask for further clarification."""

prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            f"""{system_message}\nConversation context:"""
        ),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [25]:
# Instantiate a agent and agent executor

from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)

verbose = True

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=verbose)


In [26]:
# Example

agent_executor.invoke(
    {
        "input": "tell me the most famous painting of the Jewish people"
    }
)




> Entering new AgentExecutor chain...


<-----
Prompt being sent to the LLM API:
System: You are an AI assistant specialized in providing detailed information from the Ben Uri Gallery and Museum database. Your task is to use the 'search_ben_uri_gallery_and_museum' function to help users—students, researchers, academics, critics, art market professionals, and collectors find accurate content and answers related to Jewish, Refugee, and Immigrant contributions to British visual culture. If you are not sure whether you should use the tool or respond based on your internal knowledge, ask for further clarification.
Conversation context:
Human: tell me the most famous painting of the Jewish people
----->


The concept of the "most famous painting of the Jewish people" can be interpreted in various ways, as it could refer to artworks that depict Jewish life, culture, or history, or works created by Jewish artists. To provide a comprehensive answer, I can search both the Ben Uri Gallery and M

{'input': 'tell me the most famous painting of the Jewish people',
 'output': 'The concept of the "most famous painting of the Jewish people" can be interpreted in various ways, as it could refer to artworks that depict Jewish life, culture, or history, or works created by Jewish artists. To provide a comprehensive answer, I can search both the Ben Uri Gallery and Museum for relevant artworks and Wikipedia for general information.\n\nWould you like me to proceed with this approach?'}

In [27]:
# Example

agent_executor.invoke(
    {
        "chat_history": [
            "Human: Hi there, my name is Luis!",
            "AI: Nice to meet you Luis! How can I assist you?"
        ],
        "input": "Do you remember my name?"
    }
)




> Entering new AgentExecutor chain...


<-----
Prompt being sent to the LLM API:
System: You are an AI assistant specialized in providing detailed information from the Ben Uri Gallery and Museum database. Your task is to use the 'search_ben_uri_gallery_and_museum' function to help users—students, researchers, academics, critics, art market professionals, and collectors find accurate content and answers related to Jewish, Refugee, and Immigrant contributions to British visual culture. If you are not sure whether you should use the tool or respond based on your internal knowledge, ask for further clarification.
Conversation context:
Human: Human: Hi there, my name is Luis!
Human: AI: Nice to meet you Luis! How can I assist you?
Human: Do you remember my name?
----->


Yes, your name is Luis. How can I assist you today?

> Finished chain.


{'chat_history': ['Human: Hi there, my name is Luis!',
  'AI: Nice to meet you Luis! How can I assist you?'],
 'input': 'Do you remember my name?',
 'output': 'Yes, your name is Luis. How can I assist you today?'}

**WARNING**: The AI message is wrongly sent as a Human message in this approach.

In [28]:
# Instantiate a Runnable to automatically manage memory

from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec

memory_storage = {}

def get_memory(user_id: str, chat_id: str) -> ChatMessageHistory:
    if (user_id, chat_id) not in memory_storage:
        memory_storage[(user_id, chat_id)] = ChatMessageHistory()
    return memory_storage[(user_id, chat_id)]

history_factory_config = [
    ConfigurableFieldSpec(
        id="user_id",
        annotation=str,
        name="User ID",
        description="Unique identifier for an user.",
        default="",
        is_shared=True,
    ),
    ConfigurableFieldSpec(
        id="chat_id",
        annotation=str,
        name="Chat ID",
        description="Unique identifier for a chat.",
        default="",
        is_shared=True,
    )
]

chat_agent_executor = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
    history_factory_config=history_factory_config
)

In [29]:
# Example

user_id = '123'
chat_id = '001'
user_inputs = [
    "Hi there, my name is Luis!",
    "Do you remember my name?"
]

for input in user_inputs:
    chat_agent_executor.invoke(
        {'input': input},
        config={
            'configurable': {'user_id': user_id, 'chat_id': chat_id}
        }
    )



> Entering new AgentExecutor chain...


<-----
Prompt being sent to the LLM API:
System: You are an AI assistant specialized in providing detailed information from the Ben Uri Gallery and Museum database. Your task is to use the 'search_ben_uri_gallery_and_museum' function to help users—students, researchers, academics, critics, art market professionals, and collectors find accurate content and answers related to Jewish, Refugee, and Immigrant contributions to British visual culture. If you are not sure whether you should use the tool or respond based on your internal knowledge, ask for further clarification.
Conversation context:
Human: Hi there, my name is Luis!
----->


Hello, Luis! How can I assist you today? Are you looking for information related to Jewish, Refugee, and Immigrant contributions to British visual culture?

> Finished chain.


> Entering new AgentExecutor chain...


<-----
Prompt being sent to the LLM API:
System: You are an AI assistant specialized in providing d

In [30]:
# Example 

user_id = '123'
chat_id = '002'
user_inputs = [
    "tell me the most famous painting of the Jewish people"
    "what are contributions of Refugee to British visual culture?",
]

for input in user_inputs:
    chat_agent_executor.invoke(
        {'input': input},
        config={
            'configurable': {'user_id': user_id, 'chat_id': chat_id}
        }
    )



> Entering new AgentExecutor chain...


<-----
Prompt being sent to the LLM API:
System: You are an AI assistant specialized in providing detailed information from the Ben Uri Gallery and Museum database. Your task is to use the 'search_ben_uri_gallery_and_museum' function to help users—students, researchers, academics, critics, art market professionals, and collectors find accurate content and answers related to Jewish, Refugee, and Immigrant contributions to British visual culture. If you are not sure whether you should use the tool or respond based on your internal knowledge, ask for further clarification.
Conversation context:
Human: tell me the most famous painting of the Jewish peoplewhat are contributions of Refugee to British visual culture?
----->



Invoking: `search_ben_uri_gallery_and_museum` with `{'query': 'most famous painting of the Jewish people'}`


No good information was found in Ben Uri gallery & museum
Invoking: `search_ben_uri_gallery_and_museum` with `{'query'

In [31]:
# Example 

user_id = '456'
chat_id = '001'
user_inputs = [
    "Hi, what are the contributors to artificial inteligence?"
]

for input in user_inputs:
    chat_agent_executor.invoke(
        {'input': input},
        config={
            'configurable': {'user_id': user_id, 'chat_id': chat_id}
        }
    )



> Entering new AgentExecutor chain...


<-----
Prompt being sent to the LLM API:
System: You are an AI assistant specialized in providing detailed information from the Ben Uri Gallery and Museum database. Your task is to use the 'search_ben_uri_gallery_and_museum' function to help users—students, researchers, academics, critics, art market professionals, and collectors find accurate content and answers related to Jewish, Refugee, and Immigrant contributions to British visual culture. If you are not sure whether you should use the tool or respond based on your internal knowledge, ask for further clarification.
Conversation context:
Human: Hi, what are the contributors to artificial inteligence?
----->



Invoking: `search_wikipedia` with `{'query': 'Contributors to artificial intelligence'}`


Page: Technological singularity
Summary: The technological singularity—or simply the singularity—is a hypothetical future point in time at which technological growth becomes uncontrollable and i

In [32]:
# Example

memory_storage

{('123',
  '001'): InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi there, my name is Luis!'), AIMessage(content='Hello, Luis! How can I assist you today? Are you looking for information related to Jewish, Refugee, and Immigrant contributions to British visual culture?'), HumanMessage(content='Do you remember my name?'), AIMessage(content='Yes, your name is Luis! How can I assist you today?')]),
 ('123',
  '002'): InMemoryChatMessageHistory(messages=[HumanMessage(content='tell me the most famous painting of the Jewish peoplewhat are contributions of Refugee to British visual culture?'), AIMessage(content='It appears that there were no specific results found in the Ben Uri Gallery and Museum database for the queries "most famous painting of the Jewish people" and "contributions of Refugees to British visual culture."\n\nWould you like me to try searching for this information on Wikipedia instead, or is there another specific aspect you are interested in exploring?')]),
 ('4